In [4]:
from tensorflow.keras import Input, layers, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Softmax, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
from keras.utils import np_utils
from tensorflow.keras.datasets import mnist
from keras.utils.np_utils import to_categorical
import tensorflow as tf
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = to_categorical(Y_train)
X_train = X_train.reshape(X_train.shape[0], -1)

In [5]:
n_vocabs = 256
class DotProduct(layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
  def call(self, t_queries, t_keys, t_values):
    softmax_layer = Softmax(axis = -1)
    softmax_output = softmax_layer(tf.matmul(t_queries, tf.transpose(t_keys, [0, 2, 1]))/np.sqrt(t_queries.shape[2]))
    return tf.matmul(softmax_output, t_values)

In [6]:
input = Input((X_train.shape[1]))
n_embedding = 128
qk_hidden = 256
v_hidden = 256
embedding = Embedding(n_vocabs, n_embedding, input_length = X_train.shape[1])(input)
t_queries = Dense(qk_hidden)(embedding)
t_keys = Dense(qk_hidden)(embedding)
t_values = Dense(v_hidden)(embedding)
values_probability = DotProduct()(t_queries, t_keys, t_values)
flatten = Flatten()(values_probability)
dense = Dense(128, activation = "relu")(flatten)
output = Dense(10, activation = "softmax")(dense)
model_attention = Model(input, output)
#adam_optimizer = Adam(learning_rate = 1e-4)
model_attention.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["acc"])
with tf.device("/gpu:0"):
  model_attention.fit(X_train, Y_train, batch_size = 32, epochs = 30, verbose = 1)

Epoch 1/30
1875/1875 [==============================] - 64s 32ms/step - loss: 0.4609 - acc: 0.8472
Epoch 2/30
1875/1875 [==============================] - 62s 33ms/step - loss: 0.1678 - acc: 0.9495
Epoch 3/30
1875/1875 [==============================] - 64s 34ms/step - loss: 0.1433 - acc: 0.9564
Epoch 4/30
1875/1875 [==============================] - 65s 35ms/step - loss: 0.1271 - acc: 0.9612
Epoch 5/30
1875/1875 [==============================] - 66s 35ms/step - loss: 0.1152 - acc: 0.9659
Epoch 6/30
1875/1875 [==============================] - 66s 35ms/step - loss: 0.1052 - acc: 0.9680
Epoch 7/30
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0895 - acc: 0.9725
Epoch 8/30
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0815 - acc: 0.9749
Epoch 9/30
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0702 - acc: 0.9779
Epoch 10/30
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0658 - acc: 0.9795
Epoch 11/

In [7]:
#Nếu k dùng attention layer phía trước mà dùng các pixels ảnh làm feature luôn.
input = Input((X_train.shape[1]))
dense1 = Dense(128, activation = "relu")(input)
output = Dense(10, activation = "softmax")(dense1)
model_mlp = Model(input, output)
model_mlp.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["acc"])
with tf.device("/gpu:0"):
  model_mlp.fit(X_train, Y_train, batch_size = 32, epochs = 30, verbose = 1)
#Hiệu năng thấp hơn là training với Attention Layer

Epoch 1/30
1875/1875 [==============================] - 5s 3ms/step - loss: 2.2348 - acc: 0.8658
Epoch 2/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3382 - acc: 0.9168
Epoch 3/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2723 - acc: 0.9315
Epoch 4/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2490 - acc: 0.9384
Epoch 5/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2354 - acc: 0.9402
Epoch 6/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2182 - acc: 0.9445
Epoch 7/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2044 - acc: 0.9477
Epoch 8/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1945 - acc: 0.9510
Epoch 9/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1920 - acc: 0.9530
Epoch 10/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1852 - acc: 0.9549
Epoch 11/30
1875/1875 [======

In [8]:
(X_test, Y_test) = mnist.load_data()[1]
X_test = X_test.reshape(X_test.shape[0], -1)
Y_test = np_utils.to_categorical(Y_test)
model_attention.evaluate(X_test, Y_test)
model_mlp.evaluate(X_test, Y_test)
#Dùng thêm Attention Layer có hiệu năng tốt hơn trên tập test
#Tuy nhiên thời gian training dùng thêm attention layer rất lâu ! dù đã có GPU của colab

313/313 [==============================] - 1s 3ms/step - loss: 0.4464 - acc: 0.9440


[0.4464186131954193, 0.9440000057220459]